In [1]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import joblib
import cv2
from pathlib import Path
import mediapipe as mp

In [ ]:
model = SGDClassifier(loss='hinge')
label_encoder = LabelEncoder()
df = pd.read_csv('asl_landmarks.csv')

X = df.drop('Label', axis=1)
y = label_encoder.fit_transform(df["Label"])
model.fit(X, y)

joblib.dump(model, "svm_model.pkl")
joblib.dump(label_encoder, "label-encoder.pkl")

In [2]:
def get_img():
    path = Path("../Image")

    for file in path.glob("*"):
        if file.is_file():
            return str(file)
        
    return None

In [3]:
# Get the model and load it
model_file = "model-svm.pkl"
load_model = joblib.load(model_file)

label_file = "label-encoder.pkl"
encoder = joblib.load(label_file)

# Inicialize o MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to extract landmarks from a new image and make prediction
def predict_letter(image_path, model):
    
    image = cv2.imread(image_path)
    
    if image is None:
        return None

    # Process image to get the landmarks
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Known if the hand is detected
    if results.multi_hand_landmarks:
        
        # Extract the landmarks
        landmarks = results.multi_hand_landmarks[0]

        # Format the coordinates
        landmark_data = [coord for lm in landmarks.landmark for coord in (lm.x, lm.y)]
        
        # Make prediction with the model
        prediction = model.predict([landmark_data])
        
        predicted_letter = encoder.inverse_transform(prediction)
        
        return predicted_letter[0]  # Retorna a letra prevista
    else:
        return "Error: There's no hand on the image"

image_path = get_img()
predicted_letter = predict_letter(image_path, load_model)
print(f"The predicted letter is: {predicted_letter}")


The predicted letter is: G


c:\Users\disrct\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\disrct\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(
